# Extended repressilator

This repressilator includes additional "steps" - signalling molecules that act as intermediates between repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=0.0003)
tetr = lc.Regulator(name='TetR', degradation_rate=0.0011)
ci = lc.Regulator(name='cI', degradation_rate=0.0011)

# Signalling molecules
c4 = lc.Regulator(name='C4', degradation_rate=0.0002, init_concentration=100)
scb2 = lc.Regulator(name='SCB2', degradation_rate=0.0002)
c14 = lc.Regulator(name='C14', degradation_rate=0.0002)

genetic_n.add_regulator([tetr, laci, ci, c4, scb2, c14])

TF degradation rates were taken from https://2011.igem.org/Team:ETH_Zurich/Modeling/Parameters


Degradation rate for C4-HSL and 3OHC14-HSL have been taken from supplementary data of <div class="csl-entry">Pai, A., &#38; You, L. (2009). Optimal tuning of bacterial sensing potential. <i>Molecular Systems Biology</i>, <i>5</i>, 286. https://doi.org/10.1038/MSB.2009.43</div>

and converted from h^-1 to s^-1: 
- C4-HSL 0.0133 h^-1 = 3.69*10^-6 s^-1
- 3OHC14-HSL 0.017 h^-1 = 0.0002 s^-1

Based on information here http://www.systemsbiology.ls.manchester.ac.uk/wiki/index.php/Degradation_of_C I estimate degradation rate of SCB2 to be the same as other signalling molecules.

In [ ]:
# option with producers
# rhli = lc.Regulator(name='RhlI', degradation_rate=1)
# scbb = lc.Regulator(name='ScbB', degradation_rate=1)
# cini = lc.Regulator(name='CinI', degradation_rate=1)
# genetic_n.add_regulator([rhli, scbb, cini])

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=6.16*10**-5, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=6.16*10**-5, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=6.16*10**-5, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

I initially planned to use converted half-lives of YFP and CFP from here <div class="csl-entry">Corre, G., Stockholm, D., Arnaud, O., Kaneko, G., Viñuelas, J., Yamagata, Y., Neildez-Nguyen, T. M. A., Kupiec, J. J., Beslon, G., Gandrillon, O., Paldi, A., &#38; Macarthur, B. D. (2014). Stochastic Fluctuations and Distributed Control of Gene Expression Impact Cellular Memory. <i>PLOS ONE</i>, <i>9</i>(12), e115574. https://doi.org/10.1371/JOURNAL.PONE.0115574</div>

to constant degradation rate assuming that it's a first-order reaction. I first converted h^-1 to s^-1 and then divided 0.693 by the number I got.

But experimentally they last too long in the cell for me to see how transcription factors affect expression of them. So I took degradation rate of GFP with degradation tags from here <div class="csl-entry">Andersen, J. B., Sternberg, C., Poulsen, L. K., Bjørn, S. P., Givskov, M., &#38; Molin, S. (1998). New Unstable Variants of Green Fluorescent Protein for Studies of Transient Gene Expression in Bacteria. <i>Applied and Environmental Microbiology</i>, <i>64</i>(6), 2240. https://doi.org/10.1128/AEM.64.6.2240-2246.1998</div>

converted to s^-1 and assumed that the rate is the same for all the fluorescent proteins.

In [ ]:
#First extended node
c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[0.1, 10000], K=10, n=2)
laci_not_scb2 = lc.Hill1(name='Plac', input=laci, output=[scb2, mcherry], alpha=[10000,0.1], K=10, n=2)

# Second
scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[0.1,10000], K=10, n=2)
tetr_not_c14 = lc.Hill1(name='Ptet', input=tetr, output=[c14, mven], alpha=[10000,0.1], K=10, n=2)

# Third
c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[0.1, 10000], K=10, n=2)
ci_not_c4 = lc.Hill1(name='PcI', input=ci, output=[c4, cyan], alpha=[10000,0.1], K=10, n=2)


genetic_n.add_operator([c4_laci, laci_not_scb2, scb2_tetr, tetr_not_c14, c14_ci, ci_not_c4])

In [ ]:
# # option with producers

# #First extended node
# c4_laci = lc.Hill1(name='Prhl', input=c4, output=laci, alpha=[0.1, 10000], K=10, n=2)
# laci_not_scbb = lc.Hill1(name='Plac', input=laci, output=[scbb, mcherry], alpha=[10000,0.1], K=10, n=2)
# scbb_scb2 = lc.Producer(name='ScbB', input=scbb, output=scb2, k1a=1, k2=1)

# # Second
# scb2_tetr = lc.Hill1(name='Pscb', input=scb2, output=tetr, alpha=[0.1,10000], K=10, n=2)
# tetr_not_cini = lc.Hill1(name='Ptet', input=tetr, output=[cini, mven], alpha=[10000,0.1], K=10, n=2)
# cini_c14 = lc.Producer(name='CinI', input=cini, output=c14, k1a=1, k2=1)

# # Third
# c14_ci = lc.Hill1(name='Pcin', input=c14, output=ci, alpha=[0.1, 10000], K=10, n=2)
# ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=[rhli, cyan], alpha=[10000,0.1], K=10, n=2)
# rhli_c4 = lc.Producer(name='ScbB', input=rhli, output=c4, k1a=1, k2=1)


# genetic_n.add_operator([c4_laci, laci_not_scbb, scbb_scb2, scb2_tetr, tetr_not_cini, cini_c14, c14_ci, ci_not_rhli, rhli_c4])

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw()

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(stochastic=False, track_all=True)

In [ ]:
m = assay.measurements
fig,(ax, ax1, ax2) = plt.subplots(3,1, sharex=True)
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='SCB2 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax1.legend(['LacI', 'TetR', 'cI'])
ax2.legend(['C4', 'SCB2', 'C14'])

Then I can also change how each signal diffuses (to see how that affects the system)